In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torch.optim.lr_scheduler import StepLR
from torchvision.utils import make_grid, save_image
from torchmetrics.image.fid import FrechetInceptionDistance
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import random
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
from sklearn.manifold import TSNE
import seaborn as sns
import wandb
import imageio

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device Used: {device}")

Device Used: cuda


In [3]:
saved_model_folder = "./saved_models"
os.makedirs(saved_model_folder, exist_ok=True)

In [4]:
class AFHQDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        root_dir: path of the parent directory that contains images.
        transforms: augmentations applied to the images (can be none or more).
        """
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.class_mapping = {}
        
        extensions = (".jpg", ".jpeg", ".png")
        # go through all sub-directories
        for label, category in enumerate(sorted(os.listdir(root_dir))):
            full_path = os.path.join(root_dir, category)
            if os.path.exists(full_path):
                self.class_mapping[label] = category
                for img_name in os.listdir(full_path):
                    if img_name.endswith(extensions):
                        self.image_paths.append(os.path.join(full_path, img_name))
                        self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [5]:
# Dataset Hyperparameters
img_size = 64
batch_size = 64

# dataset paths
train_dir = '/home/user/javeda1/stargan-v2/data/afhq/train'
val_dir = '/home/user/javeda1/stargan-v2/data/afhq/val'

In [6]:
transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [7]:
# Load the train and val dataset
train_dataset = AFHQDataset(root_dir=train_dir, transform=transform)
val_dataset = AFHQDataset(root_dir=val_dir, transform=transform)

# DataLoaders for train and val sets
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

train_size = len(train_loader.dataset)
val_size = len(val_loader.dataset)

print(f"Train dataset size: {train_size}")
print(f"Validation dataset size: {val_size}")

Train dataset size: 14630
Validation dataset size: 1500


In [8]:
# Define the FID computation function
def compute_fid_score_gan(generator, data_loader, device, latent_dim=256, max_samples=1000):
    """
    Computes the Fréchet Inception Distance (FID) between real and generated images.
    """
    fid = FrechetInceptionDistance(feature=2048).to(device)
    generator.eval()
    
    total_samples = 0
    with torch.no_grad():
        for real_images, _ in data_loader:
            batch_size = real_images.size(0)
            real_images = real_images.to(device)
            
            # Generate fake images
            noise = torch.randn(batch_size, latent_dim, device=device)
            fake_images = generator(noise)
            
            # Convert images to uint8 and scale to [0, 255]
            real_images = (real_images * 0.5 + 0.5).clamp(0, 1)
            fake_images = (fake_images * 0.5 + 0.5).clamp(0, 1)
            real_images = (real_images * 255).byte()
            fake_images = (fake_images * 255).byte()
            
            fid.update(real_images, real=True)
            fid.update(fake_images, real=False)
            
            total_samples += batch_size
            if total_samples >= max_samples:
                break
    
    return fid.compute().item()

In [9]:
# Define the training loop
def train_dcgan(generator, discriminator, dataloader, latent_dim, epochs, device, lr_g=0.0002, lr_d=0.0008, beta1=0.5, step_size=10, gamma=0.5,
               project="DCGAN-AFHQ2", name="run1"):
    # Initialize optimizers
    optimizer_g = optim.Adam(generator.parameters(), lr=lr_g, betas=(beta1, 0.999))
    optimizer_d = optim.Adam(discriminator.parameters(), lr=lr_d, betas=(beta1, 0.999))

    # Learning rate schedulers
    scheduler_g = optim.lr_scheduler.StepLR(optimizer_g, step_size=step_size, gamma=gamma)
    scheduler_d = optim.lr_scheduler.StepLR(optimizer_d, step_size=step_size, gamma=gamma)

    # Loss function
    criterion = nn.BCELoss()

    # Fixed noise for evaluation
    fixed_noise = torch.randn(64, latent_dim, device=device)

    # Labels for real and fake data
    real_label = 0.9  # Label smoothing for real images
    fake_label = 0.0

    # Initialize WandB
    wandb.init(project=project, name=name,reinit=False, config={
        "latent_dim": latent_dim,
        "epochs": epochs,
        "lr_g": lr_g,
        "lr_d": lr_d,
        "beta1": beta1,
        "step_size": step_size,
        "gamma": gamma
    })

    # Output directory for generated images
    output_dir = "generated_images"
    os.makedirs(output_dir, exist_ok=True)

    # Training loop
    for epoch in range(epochs):
        g_loss_total = 0.0
        d_loss_total = 0.0

        for i, (real_images, _) in enumerate(tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}")):
            batch_size = real_images.size(0)

            real_images = real_images.to(device)

            # Train Discriminator
            if i % 3 == 0:  # Train the discriminator less frequently
                discriminator.zero_grad()

                # Real images
                labels_real = torch.full((batch_size,), real_label, device=device)
                output_real = discriminator(real_images).view(-1)
                loss_real = criterion(output_real, labels_real)

                # Fake images
                noise = torch.randn(batch_size, latent_dim, device=device)
                fake_images = generator(noise)
                labels_fake = torch.full((batch_size,), fake_label, device=device)
                output_fake = discriminator(fake_images.detach()).view(-1)
                loss_fake = criterion(output_fake, labels_fake)

                # Combine losses and backpropagate
                d_loss = loss_real + loss_fake
                d_loss.backward()
                optimizer_d.step()

                d_loss_total += d_loss.item()

            # Train Generator
            generator.zero_grad()
            noise = torch.randn(batch_size, latent_dim, device=device)
            fake_images = generator(noise)
            output = discriminator(fake_images).view(-1)
            g_loss = criterion(output, labels_real)
            g_loss.backward()
            optimizer_g.step()

            g_loss_total += g_loss.item()

        # Step learning rate schedulers
        scheduler_g.step()
        scheduler_d.step()

        # Log learning rates
        # wandb.log({"Learning Rate Generator": scheduler_g.get_last_lr()[0],
        #            "Learning Rate Discriminator": scheduler_d.get_last_lr()[0]})

        # Log losses and generate sample images
        avg_g_loss = g_loss_total / len(dataloader)
        avg_d_loss = d_loss_total / len(dataloader)

        # Compute FID score
        fid_score = compute_fid_score_gan(generator, dataloader, device, latent_dim)

        # Log metrics
        # wandb.log({"Generator Loss": avg_g_loss, "Discriminator Loss": avg_d_loss, "FID Score": fid_score})

        # Log and save generated images as grids
        with torch.no_grad():
            sample_images = generator(fixed_noise).detach().cpu()
            sample_grid = make_grid(sample_images, nrow=8, normalize=True, scale_each=True)

            real_images_grid = make_grid(real_images[:64], nrow=8, normalize=True, scale_each=True)

            # Save generated grid to disk
            save_image(sample_grid, os.path.join(output_dir, f"epoch_{epoch+1:03d}_generated.png"))

        wandb.log({
            "Generated Images Grid": wandb.Image(sample_grid, caption="Generated Images"),
            "Real Images Grid": wandb.Image(real_images_grid, caption="Real Images"),
            "Learning Rate Generator": scheduler_g.get_last_lr()[0],
            "Learning Rate Discriminator": scheduler_d.get_last_lr()[0],
            "Generator Loss": avg_g_loss, "Discriminator Loss": avg_d_loss, "FID Score": fid_score
        })

    # Finish the WandB run
    wandb.finish()

In [10]:
# Define weights initialization function
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [11]:
class Generator(nn.Module):
    def __init__(self, latent_dim=256, num_channels=3):
        super(Generator, self).__init__()
        
        # Initial dense projection from latent space
        self.fc = nn.Linear(latent_dim, 512 * 4 * 4)
        self.bn1 = nn.BatchNorm2d(512)
        self.relu1 = nn.ReLU(True)
        
        # Second block: 512 x 4 x 4 -> 256 x 8 x 8
        self.upsample2 = nn.Upsample(scale_factor=2)
        self.conv2 = nn.Conv2d(512, 256, 3, 1, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(256)
        self.relu2 = nn.ReLU(True)
        
        # Third block: 256 x 8 x 8 -> 128 x 16 x 16
        self.upsample3 = nn.Upsample(scale_factor=2)
        self.conv3 = nn.Conv2d(256, 128, 3, 1, 1, bias=False)
        self.bn3 = nn.BatchNorm2d(128)
        self.relu3 = nn.ReLU(True)
        
        # Fourth block: 128 x 16 x 16 -> 64 x 32 x 32
        self.upsample4 = nn.Upsample(scale_factor=2)
        self.conv4 = nn.Conv2d(128, 64, 3, 1, 1, bias=False)
        self.bn4 = nn.BatchNorm2d(64)
        self.relu4 = nn.ReLU(True)
        
        # Final block: 64 x 32 x 32 -> num_channels x 64 x 64
        self.upsample5 = nn.Upsample(scale_factor=2)
        self.conv5 = nn.Conv2d(64, num_channels, 3, 1, 1, bias=False)
        self.tanh = nn.Tanh()

    def forward(self, x):
        # Initial projection and reshape
        x = self.fc(x.view(-1, x.size(1)))
        x = x.view(-1, 512, 4, 4)
        x = self.bn1(x)
        x = self.relu1(x)
        
        # Second block
        x = self.upsample2(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        
        # Third block
        x = self.upsample3(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu3(x)
        
        # Fourth block
        x = self.upsample4(x)
        x = self.conv4(x)
        x = self.bn4(x)
        x = self.relu4(x)
        
        # Final block
        x = self.upsample5(x)
        x = self.conv5(x)
        x = self.tanh(x)
        
        return x

class Discriminator(nn.Module):
    def __init__(self, num_channels=3):
        super(Discriminator, self).__init__()
        
        # First convolution block
        self.conv1 = nn.Conv2d(num_channels, 64, 4, 2, 1, bias=False)
        self.lrelu1 = nn.LeakyReLU(0.2, inplace=True)
        
        # Second convolution block
        self.conv2 = nn.Conv2d(64, 128, 4, 2, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(128)
        self.lrelu2 = nn.LeakyReLU(0.2, inplace=True)
        
        # Third convolution block
        self.conv3 = nn.Conv2d(128, 256, 4, 2, 1, bias=False)
        self.bn3 = nn.BatchNorm2d(256)
        self.lrelu3 = nn.LeakyReLU(0.2, inplace=True)
        
        # Fourth convolution block
        self.conv4 = nn.Conv2d(256, 512, 4, 2, 1, bias=False)
        self.bn4 = nn.BatchNorm2d(512)
        self.lrelu4 = nn.LeakyReLU(0.2, inplace=True)
        
        # Final convolution
        self.conv5 = nn.Conv2d(512, 1, 4, 1, 0, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.conv1(x)
        x = self.lrelu1(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.lrelu2(x)
        
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.lrelu3(x)
        
        x = self.conv4(x)
        x = self.bn4(x)
        x = self.lrelu4(x)
        
        x = self.conv5(x)
        x = self.sigmoid(x)
        
        return x.view(-1)

In [13]:
# Train the model
latent_dim = 128
epochs = 40
lr_g=0.0002
lr_d=0.0004
generator = Generator(latent_dim=latent_dim).to(device)
generator.apply(weights_init)
discriminator = Discriminator().to(device)
discriminator.apply(weights_init)
train_dcgan(generator, discriminator, train_loader, latent_dim, epochs, device, lr_g=lr_g, lr_d=lr_d, name='run2')

Epoch 35/40: 100%|████████████████████████████████████████| 229/229 [00:09<00:00, 24.32it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 338.00 MiB. GPU 0 has a total capacity of 23.58 GiB of which 318.62 MiB is free. Process 1753384 has 2.60 GiB memory in use. Process 86073 has 5.46 GiB memory in use. Process 158408 has 2.46 GiB memory in use. Process 223655 has 5.55 GiB memory in use. Including non-PyTorch memory, this process has 6.46 GiB memory in use. Process 216647 has 360.00 MiB memory in use. Of the allocated memory 5.79 GiB is allocated by PyTorch, and 365.91 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [12]:
# Train the model
latent_dim = 256
epochs = 40
lr_g=0.0002
lr_d=0.0004
generator = Generator(latent_dim=latent_dim).to(device)
discriminator = Discriminator().to(device)
train_dcgan(generator, discriminator, train_loader, latent_dim, epochs, device, lr_g=lr_g, lr_d=lr_d, name='run3')

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ahmadjaved97. Use `wandb login --relogin` to force relogin


Epoch 40/40: 100%|████████████████████████████████████████| 229/229 [00:11<00:00, 19.66it/s]


Discriminator Loss,█▄▄▄▄▄▃▃▃▃▂▂▂▂▂▂▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
FID Score,▃▄▄▅▅▄▄▃▄▄▃▃▃▃▂▁▃▁▃▁▁▃▁▃▃▃▅▃▄▄▅▄▅▅▂█▄▂▆▅
Generator Loss,▁▁▁▁▁▁▂▂▃▃▂▂▃▄▄▄▄▄▄▄▃▄▅▅▆▆██▇█▆▆▆▆▇██▇▇▇
Learning Rate Discriminator,█████████▄▄▄▄▄▄▄▄▄▄▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
Learning Rate Generator,█████████▄▄▄▄▄▄▄▄▄▄▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
Discriminator Loss,0.13249
FID Score,273.90256
Generator Loss,4.19751
Learning Rate Discriminator,3e-05
Learning Rate Generator,1e-05


In [15]:
# Train the model
latent_dim = 100
epochs = 50
lr_g=0.0001
lr_d=0.0003
generator = Generator(latent_dim=latent_dim).to(device)
discriminator = Discriminator().to(device)
train_dcgan(generator, discriminator, train_loader, latent_dim, epochs, device, lr_g=lr_g, lr_d=lr_d, name='run6')

Epoch 50/50: 100%|████████████████████████████████████████| 229/229 [00:08<00:00, 25.60it/s]


Discriminator Loss,█▅▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
FID Score,▆▇█▇▆▇▇▇▇▇▆▆▇▆▆▄▄▄▃▃▃▅▄▄▁▃▄▅▄▃▅▃▂▁▄▂▃▃▃▄
Generator Loss,▃▁▂▄▆▅▅▅▅▄▆▆▇▆▇▅▅▆▆▆▇▇▇▇▇▇▇▇████▇▇▇▇▇███
Learning Rate Discriminator,████████▄▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
Learning Rate Generator,████████▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
Discriminator Loss,0.1338
FID Score,197.31503
Generator Loss,3.77325
Learning Rate Discriminator,1e-05
Learning Rate Generator,0.0
